<a href="https://colab.research.google.com/github/MicroTee/fem-optimization-template/blob/main/%5BNCS%5D_project_buggy_TABU_SEARCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Getting Started**

In [198]:
!python --version

Python 3.11.11


In [199]:
# %%time
# %%capture

# # !pip install openseespy pyswarms
# !pip install numba
# !pip install SciencePlots==1.0.9

In [200]:
import numpy as np
import pandas as pd
# import seaborn as sns

from numba import njit # decor chuyển mã Python sang mã máy
from numba.typed import List # Tạo mảng động với Numba
from numba import types as nbtypes # kiểu dữ liệu Numba

# import openseespy.opensees as ops
# from pyswarms import pso

import math
import random
import time
import matplotlib.pyplot as plt

**Start time counter**

In [201]:
startTime = time.time()

# **Declaration of Input Data**

Khởi tạo

Read data

In [202]:
def load_knapsack_data():
    info_path = "/content/knapPI_2_500_1000_1_info.csv"
    item_path = "/content/knapPI_2_500_1000_1_items.csv"

    items_df = pd.read_csv(item_path, \
                           header=0)
    info_df = pd.read_csv(info_path, \
                          header=None)
    values = items_df[" price"].to_numpy()
    weights = items_df[" weight"].to_numpy()
    answer_solution = items_df[" sol"].to_numpy()
    N = info_df.iloc[0,1] # số món đồ
    capacity = info_df.iloc[1,1] # sức chứa
    answer_value = info_df.iloc[2,1] # lời giải

    return N, weights, values, capacity, answer_solution, answer_value

In [203]:
N, weights, values, capacity, answer_solution, answer_value = load_knapsack_data()

# Cố định seed để kết quả tái hiện
np.random.seed(0)

# Số món đồ
# N = 100

# Sinh weights ngẫu nhiên trong khoảng [1, 20)
# weights = np.random.randint(1, 20, size=N)

# Sinh values ngẫu nhiên trong khoảng [1, 100)
# values = np.random.randint(1, 100, size=N)

# Đặt capacity bằng một nửa tổng trọng lượng
# capacity = int(np.sum(weights) / 2)

# vòng lặp tối đa
max_interation = 1000

# số lượt cấm
tabu_tenure = 5

no_move_counter = 0
max_no_move = 10

# **Declaration of Output Data**

# **Declaration of Function**

## **Hàm mục tiêu**

Trả về giá trị và trọng lượng

In [204]:
@njit
def evaluate_knapsack(solution, weights, values, capacity):
    """
        Hàm tính tổng giá trị và trọng lượng của nghiệm:
        - solution: mảng nhị phân biểu thị món đồ được chọn
        - weights, values: mảng trọng lượng, giá trị tương ứng
        - capacity: khối lượng tối đa có thể xếp
        Trả về:
        - total_value: tổng giá trị
        - total_weight: tổng trọng lượng
    """

    total_value = 0
    total_weight = 0
    n = len(solution)

    for i in range(n):
        if solution[i] == 1:
            total_value += values[i]
            total_weight += weights[i]

    return total_value, total_weight


Khởi tạo

In [205]:
@njit
def generate_initial_solution(weights, values, capacity):
    """
    Hàm tạo nghiệm ban đầu:
    - weights, values: mảng trọng lượng, giá trị tương ứng
    - capacity: khối lượng tối đa có thể xếp
    Trả về:
    - solution: mảng nhị phân biểu thị món đồ được chọn
    """

    n = len(weights)
    solution = np.zeros(n, dtype=np.int32)

    # Thêm món đồ ngẫu nhiên nếu còn dư sức chứa
    indices = np.random.permutation(n)
    total_weight = 0
    for i in indices:
        if total_weight + weights[i] <= capacity:
            solution[i] = 1
            total_weight += weights[i]
            # thoát vòng lặp khi khi đạt khối lượng tối đa
            if total_weight == capacity:
                break

    return solution

Tìm nghiệm lân cận

In [206]:
@njit
def get_neighbors(solution, iteration):
    """
    Hàm lấy tất cả các nghiệm lân cận hiện tại bằng cách lật 1 bit duy nhất
    Hàm tìm các nghiệm lân cận:
    - solution: mảng nhị phân biểu thị món đồ được chọn
    Trả về:
    - neighbors: danh sách các nghiệm lân cận
    """

    n = len(solution)
    k = iteration
    d = np.array([2,2,5,3])
    # Sử dụng numba list
    neighbors = List.empty_list(item_type=nbtypes.int32[:])
    # mỗi phần tử là 1 vector (các bit lật)
    flipped_bits_list = List.empty_list(nbtypes.int32[:])

    if k % d[0] == 0:
        for _ in range(int(n)):
            neighbor = solution.copy()
            # chọn 10 bits ngẫu nhiên để lật
            bits = np.random.choice(n, d[0], replace=False)
            for b in bits:
                neighbor[b] = 1 - neighbor[b]
            # Đưa vào List của numba: bits cần kiểu int32[:]
            bits_array = bits.astype(np.int32)

            flipped_bits_list.append(bits_array)
            neighbors.append(neighbor)
    # elif k % d[1] == 0:
    #     for _ in range(int(n/d[1])):
    #         neighbor = solution.copy()
    #         # chọn 10 bits ngẫu nhiên để lật
    #         bits = np.random.choice(n, d[1], replace=False)
    #         for b in bits:
    #             neighbor[b] = 1 - neighbor[b]
    #         # Đưa vào List của numba: bits cần kiểu int32[:]
    #         bits_array = bits.astype(np.int32)

    #         flipped_bits_list.append(bits_array)
    #         neighbors.append(neighbor)
    else:
        for i in range(n):
            neighbor = solution.copy()
            neighbor[i] = 1 - neighbor[i]

            flipped_bits_list.append(np.array([i], dtype=np.int32))
            neighbors.append(neighbor)

    return neighbors, flipped_bits_list

## **Hàm ràng buộc**

Ràng buộc về trọng lượng

In [207]:
@njit
def qualified_capacity(weight, capacity):
    return weight <= capacity


Các ràng buộc về điều kiện biên trong miền tìm kiếm

Xấp xỉ

## **Hàm phân tính PP PTHH**

## **Hàm Metaheuristic**

In [208]:
@njit
def tabu_search(weights, values, capacity, \
                max_interation, tabu_tenure, \
                no_move_counter, max_no_move):
    """
    Hàm chính triển khai Tabu Search
    Thực hiện Tabu Search cho bài toán 0-1 Knapsack.
    - weights, values, capacity: dữ liệu bài toán
    - max_iterations: số vòng lặp tối đa
    - tabu_tenure: kích thước danh sách Tabu

    Trả về: (best_solution, best_value)
    """

    # 1. Khởi tạo
    current_solution = generate_initial_solution(weights, values, capacity)
    best_solution = current_solution.copy()
    best_value, _ = evaluate_knapsack(best_solution, weights, values, capacity)

    # Lưu tabu_list dưới dạng mảng của numba
    # mô tả vị trí bit lật và thời điểm hết tabu
    n = len(weights)
    tabu_list = np.zeros(n, dtype=np.int32)

    iteration = 0

    # 2. Bắt đầu vòng lặp chính
    while iteration < max_interation:
        iteration += 1

        # lấy danh sách nghiệm lân cận
        neighbors, flipped_bits_list = get_neighbors(current_solution, iteration)

        # tìm nghiệm lân cận tốt nhất hợp lệ
        best_neighbor_value = -1
        best_neighbor = None
        best_neighbor_bits = None # theo dõi các bit lật

        # đánh giá tất cả các nghiệm lân cận
        for i, neighbor in enumerate(neighbors):
            neighbor_value, neighbor_weight = \
                evaluate_knapsack(neighbor, weights, values, capacity)
            # bit bị lật là i, do hàm get_neighbors lật từng bit theo thứ tự
            bit_flipped = flipped_bits_list[i]

            # kiểm tra ràng buộc
            if not qualified_capacity(neighbor_weight, capacity):
                continue # do không hợp lệ

            # kiểm tra tabu: nếu bit_flipped vẫn đang bị cấm, ta bỏ qua
            # trừ khi nghiệm này tốt hơn best solution toàn cục => aspiration
            is_tabu = False
            for bits in bit_flipped:
                if tabu_list[bits] > iteration:
                    # move này đang bị cấm, trừ khi asp cải thiện best_value
                    if neighbor_value <= best_value:
                        is_tabu = True
                        break

            if is_tabu:
                continue # bỏ qua nghiệm này do bị cấm

            # trường hợp từ đây trở xuống là hợp lệ
            # trường hợp có giá trị cao hơn, cập nhật:
            if neighbor_value > best_neighbor_value:
                best_neighbor_value = neighbor_value
                best_neighbor = neighbor
                best_neighbor_bits = bit_flipped

        # nếu không có nghiệm cải thiện:
        if best_neighbor is None:
            no_move_counter += 1
            if no_move_counter >= max_no_move:
                print("break at", iteration)
                break # khả năng xảy ra tuy ít, nhưng với bài toán "cực đoan"
                # vẫn hoàn toàn xuất hiện
            else:
                # khởi tạo lại
                current_solution = generate_initial_solution(weights, values, capacity)
                # đánh giá lại
                tmp_value, tmp_weight = evaluate_knapsack(current_solution, weights, values, capacity)
                # cập nhật nghiệm tốt nhất nếu tìm thấy
                if (tmp_value > best_value) and (qualified_capacity(tmp_weight, capacity)):
                    best_value = tmp_value
                    best_solution = current_solution.copy()
                    no_move_counter = 0
                # reset vì ta đã "phá vỡ" bế tắc bằng 1 nghiệm mới
                no_move_counter = 0
                continue
        else:
            current_solution = best_neighbor.copy()

        # 3. Cập nhật nghiệm hiện tại
        # đánh dấu move vừa thực hiện lên tabu list
        for bit in best_neighbor_bits:
                tabu_list[bit] = iteration + tabu_tenure

        # cập nhật nghiệm tốt nhất nếu được
        if best_neighbor_value > best_value:
            best_value = best_neighbor_value
            best_solution = best_neighbor.copy()
            no_move_counter = 0

    return best_solution, best_value


# **MAIN**

In [209]:
if __name__ == "__main__":

    count = 0
    max_value = 0
    for i in range(30):
        solution, value = tabu_search(weights, values, capacity, \
                                    max_interation, tabu_tenure, \
                                    no_move_counter, max_no_move)
        if value > max_value:
            max_value = value
        print(f"Best value: {value}. %: {value/answer_value*100:.2f}")

    print(f"\nMax value: {max_value}. %: {max_value/answer_value*100:.2f}")
    # print("Best solution found:", solution)
    # print("Best value:", value)
    # total_value, total_weight = evaluate_knapsack(solution, weights, values, capacity)
    # print("Check -> Total weight:", total_weight, " <= Capacity:", capacity)

Best value: 4352. %: 95.31
Best value: 4198. %: 91.94
Best value: 4211. %: 92.23
Best value: 4266. %: 93.43
Best value: 4231. %: 92.66
Best value: 4100. %: 89.79
Best value: 4143. %: 90.74
Best value: 4289. %: 93.93
Best value: 4210. %: 92.20
Best value: 4324. %: 94.70
Best value: 4177. %: 91.48
Best value: 4289. %: 93.93
Best value: 4186. %: 91.68
Best value: 4196. %: 91.90
Best value: 4173. %: 91.39
Best value: 4187. %: 91.70
Best value: 4214. %: 92.29
Best value: 4097. %: 89.73
Best value: 4156. %: 91.02
Best value: 4215. %: 92.31
Best value: 4226. %: 92.55
Best value: 4088. %: 89.53
Best value: 4163. %: 91.17
Best value: 4221. %: 92.44
Best value: 4065. %: 89.03
Best value: 4146. %: 90.80
Best value: 4202. %: 92.03
Best value: 4251. %: 93.10
Best value: 4014. %: 87.91
Best value: 4367. %: 95.64

Max value: 4367. %: 95.64


**Running time**

In [210]:
runningTime = time.time() - startTime
print(f'Running time is {runningTime:.3f} seconds')

Running time is 170.130 seconds
